In [133]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [134]:
file_blast = "/home/juan/Desktop/juan/bio/mites/data/blast/families_tgac.csv"
file_blast_out = "/home/juan/Desktop/juan/bio/mites/data/blast/families_tgac.filtered.80.csv"

In [135]:
df = pd.read_csv(file_blast, index_col=False, sep='\t', comment='#', header=None)
df.columns = ['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']
print(len(df.index))

1599699


In [136]:
df = df[df.pident > 80]
print(len(df.index))

1482719


In [137]:
#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)


In [ ]:
# filter overlapped 
my_index = 0
indexes = []
discard = []
count = 0
curr = 0
total_len = len(df.index)
padding = 10
while my_index < total_len:
    row = df.iloc[[my_index]]
    cond = True
    next_index = 1
    while cond and (my_index + next_index) <= total_len - 1:
        second_row = df.iloc[[my_index + next_index]]
        c1 = (row.iloc[0].sseqid == second_row.iloc[0].sseqid)
        c2 = (abs(second_row.iloc[0].sstart - row.iloc[0].sstart) <= padding)
        c3 = (abs(second_row.iloc[0].send - row.iloc[0].send) <= padding)
        cond =  c1 and c2 and c3
        if not cond:
#            indexes.append(my_index)
            break
        next_index += 1
    indexes.append(my_index)
    my_index += next_index
    #just a counter
    curr_new = int(my_index * 100 * 1.0 / (total_len * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 5 == 0:
            print(curr_new)
#indexes.append(total_len - 1)

In [ ]:
df = df[df.index.isin(indexes)]
df.sort_values(['sseqid', 'sstart'], inplace=True)
non_overlapped = str(len(df.index))
print('Non overlapped: ' + non_overlapped)

In [ ]:
df.to_csv(file_blast_out, index=None, sep='\t')
file_blast_out